<img src="../images/logo.jpg" style="width:85px;height:85px;float:left" />
<h1 style="position:relative;float:left;display:inline">Writing a GAN using AshPy and TensorFlow Datasets</h1>

<!--
<a href='https://colab.research.google.com/github/zurutech/gans-from-theory-to-production/blob/master/3.%20AshPy/3.1.%20AshPy%20and%20TensorFlow%20Datasets.ipynb'>
    <img align="left" src='https://cdn-images-1.medium.com/max/800/1*ZpNn76K98snC9vDiIJ6Ldw.jpeg'></img>
</a>
-->
<a href="http://mybinder.org/v2/gh/zurutech/gans-from-theory-to-production/master">
    <img align="left" src="http://mybinder.org/badge_logo.svg" />
</a>

In [1]:
! pip install --upgrade tensorflow-gpu==2.0.0beta1
# ! pip install --upgrade tensorflow==2.0.0beta1
! pip install --upgrade ashpy

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#GANs-with-AshPy-and-TensorFlow-Datasets-(tfds)" data-toc-modified-id="GANs-with-AshPy-and-TensorFlow-Datasets-(tfds)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>GANs with AshPy and TensorFlow Datasets (tfds)</a></span><ul class="toc-item"><li><span><a href="#AshPy-Essentials" data-toc-modified-id="AshPy-Essentials-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>AshPy Essentials</a></span></li><li><span><a href="#tfds-and-AshPy-input-format" data-toc-modified-id="tfds-and-AshPy-input-format-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>tfds and AshPy input format</a></span><ul class="toc-item"><li><span><a href="#Getting-the-data-ready-to-use" data-toc-modified-id="Getting-the-data-ready-to-use-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Getting the data ready to use</a></span></li></ul></li></ul></li><li><span><a href="#DCGAN-Theory-and-Practice" data-toc-modified-id="DCGAN-Theory-and-Practice-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>DCGAN Theory and Practice</a></span><ul class="toc-item"><li><span><a href="#Generator:-from-noise-to-insight" data-toc-modified-id="Generator:-from-noise-to-insight-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Generator: from noise to insight</a></span><ul class="toc-item"><li><span><a href="#Deconvolution" data-toc-modified-id="Deconvolution-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Deconvolution</a></span></li><li><span><a href="#Batch-Normalization" data-toc-modified-id="Batch-Normalization-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Batch Normalization</a></span></li></ul></li><li><span><a href="#Discriminator" data-toc-modified-id="Discriminator-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Discriminator</a></span></li><li><span><a href="#Loss-function:-a-bridge-between-two-networks" data-toc-modified-id="Loss-function:-a-bridge-between-two-networks-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Loss function: a bridge between two networks</a></span></li><li><span><a href="#Tensorboard" data-toc-modified-id="Tensorboard-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Tensorboard</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Training</a></span></li></ul></li><li><span><a href="#Towards-Serving" data-toc-modified-id="Towards-Serving-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Towards Serving</a></span></li><li><span><a href="#Links" data-toc-modified-id="Links-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Links</a></span></li></ul></div>

## GANs with AshPy and TensorFlow Datasets (tfds)

In this notebook, we are going to try to define a GAN and its input data pipeline using the AshPy and `tfds`. Our aim is to build a **face generator** where each image is a $64\times64\times3$ tensor.

### AshPy Essentials

AshPy is a TensorFlow 2.0 library for (**distributed**) training, evaluation, model selection, and fast prototyping. It is designed to ease the burden of setting up all the nuances of the architectures built to train complex custom deep learning models.

The library has been designed following the TensorFlow 2.0 principles:

- Ease of use.
- Keras models centric.

(Some of ) the features AshPy offers are:

- You can distribute your model training using any `tf.distribute` distribution strategy without worrying about anything.
- You can develop a state of the art model with high-level intuitive code.
- It is fully compatible with the TensorFlow 2.0 ecosytem.
- Custom losses and training behaviours are easy to implement by defining custom `ashpy.Executor`.
- Fully compatibile with the Keras API specification and the `tf.keras` package.
- AshPy correctly uses `tf.function` to accelerate the training loop for you.
- Automatic Model selection and model saving-restore.
- Automatic integration with TensorBoard.
- Ready to use models and losses for fast prototyping.

![bella](images/dataset-ashpy.png)

**AshPy enforces a programming style.**

Every trainer accepts a Keras model (or more than one, it depends on the training type), a dataset, the executor (loss computation behavior) and the hyperparameters and all of them must implement well-defined interfaces.

In order to correctly separate the data input pipeline from the model, let's introduce `tensorflow-datasets` (`tfds`): the collection of datasets ready to use, all built upon the `tf.data.Dataset` API.

### tfds and AshPy input format

The `tf.data` API has been designed to write complex input pipelines in a very simple manner. It uses the **named pattern idiom** (also called **method chaining**) and its methods are inspired to the functional programming languages that applies transformations to lists.

The most imporant class is the `tf.data.Dataset` class that represents a sequence of elements: can apply transformation to this sequence of elements in order to create our dataset.

Every `ashpy.Trainer` requires a `tf.data.Dataset` object to use.

When using a `ashpy.AdversarialTrainer` that, as the name suggests, implements the adversarial training process, the `tf.data.Dataset` object should be in a well-known format.

In particular, the dataset return type must always be in the format

```python
tuple(tuple(a,b), c)
```

where:

- **a** is the input sample.
- **b** is the label/condition (if any, otherwise fill it with 0).
- **c** is the generator input, usually the __noise__.

Using `tensorflow-datasets` we don't have to bother about the download, preprocess and iterator generator: for the most common dataset everything is ready to use.

Downloading, having all the information, and a `tf.data.Dataset` object ready-to-use is just a single line.

#### Getting the data ready to use

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import ashpy
from IPython import display

dataset_name = "celeb_a"
if dataset_name not in tfds.list_builders():
    raise ValueError(f"Something wrong with tfds: missing {dataset_name}")
dataset, info = tfds.load(dataset_name,split=tfds.Split.ALL, with_info=True)
print(info.features["image"])

Image(shape=(218, 178, 3), dtype=tf.uint8)


Since our goal is to generate $64\times64$ images and training on batches, we can use method chaining on the returned `dataset` object to generate an optimized training input pipeline that fits all our needs.

Moreover, we have to follow the AshPy convention and return the data in the correct format:
```python
tuple(tuple(a,b), c)
```

In [3]:
def convert_and_resize(features):
    image = tf.image.convert_image_dtype(features["image"], dtype=tf.float32)
    image = (image - 0.5) * 2
    image = tf.image.resize(image, size=(64, 64))
    features["image"] = image
    return features

# Extract a subset of the dataset (to speedup the training)
dataset = dataset.take(1000)
dataset = dataset.map(convert_and_resize)
# Now the dataset is a dataset of dictionaries
# setting it to the ashpy format.

latent_dimension = 100
dataset = dataset.map(lambda row: (
    (row["image"], 0), 
    tf.random.normal(shape=(latent_dimension,))
))

# Batch
batch_size = 32
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

Downloading, post-processing and create an optimized input pipeline is just these few lines above!

Since we're working on images, we'll use an architecture created for this purpose: DCGAN <sup>[3](#3)</sup>.

## DCGAN Theory and Practice

![DCGAN](images/dcgan.png)

If you take a look at [this impressive list of GANs [2\]](#2), you would find out that **DCGAN**, the architecture of our choice, is a small drop in the ocean. While there may be sexier ones, very few offers the same level of clarity, performance and computational efficiency. For these reasons DCGAN is considered one of the cornerstones of this field.

Alec Radford, Luke Metz and Soumith Chintala proposed the architecture in [their 2015 paper [3\]](#3). The idea behind DCGAN is quite straightforward. Combining a set of architectural constraints with the power of CNN yielded a robust, stable and competitive model. Moreover, the architecture is simple: 4 deconvolutional layers for the `Generator` and 4 convolutional layers for the `Discriminator`. The constraints are the following:

- All pooling layers are replaced with strided convolutions (discriminator) and fractionally-strided convolutions (generator).
- Batch-normalization used in both networks.
- Removal of the fully-connected layers (except for the discriminator output).
- `ReLU` for all Generator layers except the output, which uses `tanh`.
- `LeakyReLU` activation in the discriminator for all layers.

Recently there have been some advancements in state of the art (e.g., Spectral Normalization). However, it is essential to have a firm understanding of the basic concepts. So, we leave you (an opinionated) list of further resources at the end of this notebook, to get you up to speed with the most exciting researches.

### Generator: from noise to insight

![DCGAN Generator](images/dcgan_generator.png)

Recalling the theoretical explanation, the Generator is the network responsible for the data-generation. It learns how to fool the discriminator, so it learns how to produce realistic results. Those results are then "sampled" from the learned manifold.

The most common type of generator input is noise (i.e.: random values). More specialized GANs may require extra parameters. Since our full-demo uses a Deep Convolutional GAN (DCGAN), we don't need any other parameters.

#### Deconvolution

Intuitively, the idea behind this operation is the following:

![Deconvolution](images/deconvolution.png)


> When neural networks generate images, they build the images starting from low-resolution high-level descriptions. In this way, the network starts describing a 
> rough representation and then fill in the details to create the final image.
>
>To do this, we need some way to go from a lower resolution image to a higher one. We generally do this with the deconvolution operation. 
>Roughly, deconvolution layers allow the model to use the points from the small image to “paint” a larger area in the bigger output image.

In practice, the deconvolution operation is often implemented by resizing, using bi-linear or nearest neighbor interpolation, followed by a convolution operation.

#### Batch Normalization

What you need to know about batch normalization is that is a layer that normalizes the values. TensorFlow makes it very easy to implement such operation:  

```python
tf.keras.layers.BatchNormalization()
```

The benefit of using batch normalization has been extensively discussed and proved in various papers. We do not enter into the theoretical depth of BatchNormalization, but you can refer to [[3\]](#3), [[8\]](#8) and [[9\]](#9) to learn more about the topic.

In [4]:
G = tf.keras.Sequential([
    tf.keras.layers.Dense(1024*4*4, use_bias=False, input_shape=(100,)),
    tf.keras.layers.Reshape((4,4,1024)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    
    tf.keras.layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),

    tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    
    tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    
    tf.keras.layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    tf.keras.layers.Activation(tf.math.tanh)
])

### Discriminator
![DCGAN Discriminator](images/dcgan_discriminator.png)

In [5]:
D = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),

    tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    
    tf.keras.layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    
    tf.keras.layers.Conv2D(512, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    
    tf.keras.layers.Reshape((-1,)),
    tf.keras.layers.Dense(1)
])

### Loss function: a bridge between two networks

There are a lot of loss functions usable in GANs' architectures. From very domain-specific ones to others that are perfect for general use cases. Since our goal here falls in the latter category, we use the so-called **Non-Saturating Loss** which is the non-saturating variant of the **MinMax Loss** proposed by Goodfellow in the [original paper [1\]](#1).

As stated above, one of `AshPy`'s beauties is its offer of ready-to-use losses. If you cannot find the loss you want, you can create your own. 

We use the following two losses:

```python
from ashpy.losses.gan import DiscriminatorMinMax, GeneratorBCE
generator_bce = GeneratorBCE()
minmax = DiscriminatorMinMax()
```

It is that easy.

In [6]:
from ashpy.losses.gan import DiscriminatorMinMax, GeneratorBCE
generator_bce = GeneratorBCE()
minmax = DiscriminatorMinMax()

### Tensorboard

In [7]:
# Load the TensorBoard notebook extension and run it
%load_ext tensorboard
%tensorboard --logdir log

### Training

Using AshPy the training loop is already implemented and everyting is ready to be used in distributed or local fashion, witout any change.

Just define the hyperparameters, like the optimizers to use, the metrics to measure, and the number of epochs to train, and the trainer object will do everything for you.

In [8]:
from ashpy.trainers.gan import AdversarialTrainer

epochs = 300
logdir = "log"
metrics = []

trainer = AdversarialTrainer(
    generator=G,
    discriminator=D,
    generator_optimizer=tf.optimizers.Adam(1e-4),
    discriminator_optimizer=tf.optimizers.Adam(1e-4),
    generator_loss=generator_bce,
    discriminator_loss=minmax,
    epochs=epochs,
    metrics=metrics,
    logdir=logdir,
)

trainer(dataset)

Initializing checkpoint.


W0819 12:21:36.018539 140086811756160 deprecation.py:323] From /data/pgaleone/env/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:182: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[10] g_loss: 3.1023786067962646 - d_loss: 0.12607771158218384
[20] g_loss: 3.9864680767059326 - d_loss: 0.20067942142486572
[30] g_loss: 3.4156322479248047 - d_loss: 0.20362694561481476
[31] Saved checkpoint: log/ckpts/ckpt-1
Epoch 1 completed.
[40] g_loss: 3.738818645477295 - d_loss: 0.020717082545161247
[50] g_loss: 6.410630226135254 - d_loss: 0.01588129810988903
[60] g_loss: 5.2700629234313965 - d_loss: 0.03122032806277275
[62] Saved checkpoint: log/ckpts/ckpt-2
Epoch 2 completed.
[70] g_loss: 4.521557331085205 - d_loss: 0.05670560151338577
[80] g_loss: 4.899657249450684 - d_loss: 0.03549738973379135
[90] g_loss: 6.799798965454102 - d_loss: 0.07048636674880981
[93] Saved checkpoint: log/ckpts/ckpt-3
Epoch 3 completed.
[100] g_loss: 4.4114227294921875 - d_loss: 0.010578876361250877
[110] g_loss: 4.8765339851379395 - d_loss: 0.011637222021818161
[120] g_loss: 5.257278919219971 - d_loss: 0.009329179301857948
[124] Saved checkpoint: log/ckpts/ckpt-4
Epoch 4 completed.
[130] g_loss: 4.95

[1010] g_loss: 6.359277725219727 - d_loss: 0.0076248059049248695
[1020] g_loss: 5.024300575256348 - d_loss: 0.01778220571577549
[1023] Saved checkpoint: log/ckpts/ckpt-33
Epoch 33 completed.
[1030] g_loss: 4.124502182006836 - d_loss: 0.022080373018980026
[1040] g_loss: 5.012825012207031 - d_loss: 0.14577282965183258
[1050] g_loss: 11.733231544494629 - d_loss: 0.03863545507192612
[1054] Saved checkpoint: log/ckpts/ckpt-34
Epoch 34 completed.
[1060] g_loss: 6.682621002197266 - d_loss: 0.04453770071268082
[1070] g_loss: 5.718838691711426 - d_loss: 0.03886964172124863
[1080] g_loss: 5.7575459480285645 - d_loss: 0.02707757242023945
[1085] Saved checkpoint: log/ckpts/ckpt-35
Epoch 35 completed.
[1090] g_loss: 2.5985889434814453 - d_loss: 0.16201108694076538
[1100] g_loss: 17.208778381347656 - d_loss: 0.013130931183695793
[1110] g_loss: 5.5402116775512695 - d_loss: 0.09380918741226196
[1116] Saved checkpoint: log/ckpts/ckpt-36
Epoch 36 completed.
[1120] g_loss: 3.9489543437957764 - d_loss: 0.

[1990] g_loss: 7.722517013549805 - d_loss: 0.00799054466187954
[2000] g_loss: 5.72402286529541 - d_loss: 0.03791375085711479
[2010] g_loss: 7.289953231811523 - d_loss: 0.004798362031579018
[2015] Saved checkpoint: log/ckpts/ckpt-65
Epoch 65 completed.
[2020] g_loss: 8.256820678710938 - d_loss: 0.030118905007839203
[2030] g_loss: 3.9956235885620117 - d_loss: 0.048132672905921936
[2040] g_loss: 4.1065239906311035 - d_loss: 0.04828700050711632
[2046] Saved checkpoint: log/ckpts/ckpt-66
Epoch 66 completed.
[2050] g_loss: 3.3962221145629883 - d_loss: 0.06908851861953735
[2060] g_loss: 5.553679466247559 - d_loss: 0.024330569431185722
[2070] g_loss: 5.311236381530762 - d_loss: 0.019621383398771286
[2077] Saved checkpoint: log/ckpts/ckpt-67
Epoch 67 completed.
[2080] g_loss: 8.156453132629395 - d_loss: 0.041080404072999954
[2090] g_loss: 5.171710014343262 - d_loss: 0.01762758567929268
[2100] g_loss: 5.794774532318115 - d_loss: 0.051343221217393875
[2108] Saved checkpoint: log/ckpts/ckpt-68
Epo

...

[4929] Saved checkpoint: log/ckpts/ckpt-159
Epoch 159 completed.
[4930] g_loss: 3.46563720703125 - d_loss: 0.08316778391599655
[4940] g_loss: 7.359714508056641 - d_loss: 0.019024085253477097
[4950] g_loss: 4.225822448730469 - d_loss: 0.05300651118159294
[4960] g_loss: 7.210484504699707 - d_loss: 0.05447511374950409
[4960] Saved checkpoint: log/ckpts/ckpt-160
Epoch 160 completed.
[4970] g_loss: 9.52499008178711 - d_loss: 0.01345040649175644
[4980] g_loss: 7.12285852432251 - d_loss: 0.0024622613564133644
[4990] g_loss: 4.490544319152832 - d_loss: 0.07245595008134842
[4991] Saved checkpoint: log/ckpts/ckpt-161
Epoch 161 completed.
[5000] g_loss: 5.877676010131836 - d_loss: 0.031324271112680435
[5010] g_loss: 4.250940322875977 - d_loss: 0.08388318121433258
[5020] g_loss: 6.528748035430908 - d_loss: 0.012937605381011963
[5022] Saved checkpoint: log/ckpts/ckpt-162
Epoch 162 completed.
[5030] g_loss: 4.097959518432617 - d_loss: 0.07689115405082703
[5040] g_loss: 2.335324287414551 - d_loss: 0.

[5900] g_loss: 4.233189582824707 - d_loss: 0.03333602100610733
[5910] g_loss: 7.482357025146484 - d_loss: 0.01221203152090311
[5920] g_loss: 6.6174750328063965 - d_loss: 0.072931669652462
[5921] Saved checkpoint: log/ckpts/ckpt-191
Epoch 191 completed.
[5930] g_loss: 9.887803077697754 - d_loss: 0.028844282031059265
[5940] g_loss: 4.513058185577393 - d_loss: 0.041795726865530014
[5950] g_loss: 7.630817413330078 - d_loss: 0.007139756344258785
[5952] Saved checkpoint: log/ckpts/ckpt-192
Epoch 192 completed.
[5960] g_loss: 9.216154098510742 - d_loss: 0.03949076309800148
[5970] g_loss: 6.7998247146606445 - d_loss: 0.04309137165546417
[5980] g_loss: 4.9447479248046875 - d_loss: 0.019645433872938156
[5983] Saved checkpoint: log/ckpts/ckpt-193
Epoch 193 completed.
[5990] g_loss: 12.490989685058594 - d_loss: 0.4924812614917755
[6000] g_loss: 8.926315307617188 - d_loss: 0.0006283631082624197
[6010] g_loss: 8.299477577209473 - d_loss: 0.029553081840276718
[6014] Saved checkpoint: log/ckpts/ckpt-1

[6880] g_loss: 2.637939929962158 - d_loss: 0.17102864384651184
[6882] Saved checkpoint: log/ckpts/ckpt-222
Epoch 222 completed.
[6890] g_loss: 8.331697463989258 - d_loss: 0.007991527207195759
[6900] g_loss: 7.940680503845215 - d_loss: 0.006174752954393625
[6910] g_loss: 7.997971057891846 - d_loss: 0.010777242481708527
[6913] Saved checkpoint: log/ckpts/ckpt-223
Epoch 223 completed.
[6920] g_loss: 3.977123737335205 - d_loss: 0.037614502012729645
[6930] g_loss: 14.198189735412598 - d_loss: 0.0004569671000353992
[6940] g_loss: 5.598245143890381 - d_loss: 0.030772896483540535
[6944] Saved checkpoint: log/ckpts/ckpt-224
Epoch 224 completed.
[6950] g_loss: 3.909323215484619 - d_loss: 0.11536019295454025
[6960] g_loss: 6.955147743225098 - d_loss: 0.02812594175338745
[6970] g_loss: 10.403316497802734 - d_loss: 0.03512096405029297
[6975] Saved checkpoint: log/ckpts/ckpt-225
Epoch 225 completed.
[6980] g_loss: 10.794671058654785 - d_loss: 0.04833587631583214
[6990] g_loss: 8.85223388671875 - d_l

[7850] g_loss: 6.478597640991211 - d_loss: 0.0036167888902127743
[7860] g_loss: 5.468390941619873 - d_loss: 0.01150368433445692
[7870] g_loss: 5.4288153648376465 - d_loss: 0.044104814529418945
[7874] Saved checkpoint: log/ckpts/ckpt-254
Epoch 254 completed.
[7880] g_loss: 4.724697589874268 - d_loss: 0.02361968904733658
[7890] g_loss: 6.754733085632324 - d_loss: 0.01517343521118164
[7900] g_loss: 7.088025093078613 - d_loss: 0.01923716813325882
[7905] Saved checkpoint: log/ckpts/ckpt-255
Epoch 255 completed.
[7910] g_loss: 5.847674369812012 - d_loss: 0.01624368131160736
[7920] g_loss: 6.579262733459473 - d_loss: 0.00513227004557848
[7930] g_loss: 6.988648891448975 - d_loss: 0.009359386749565601
[7936] Saved checkpoint: log/ckpts/ckpt-256
Epoch 256 completed.
[7940] g_loss: 5.693020820617676 - d_loss: 0.019677959382534027
[7950] g_loss: 5.416482925415039 - d_loss: 0.01931922137737274
[7960] g_loss: 4.767340660095215 - d_loss: 0.017098788172006607
[7967] Saved checkpoint: log/ckpts/ckpt-25

[8820] g_loss: 9.410928726196289 - d_loss: 0.002459548646584153
[8830] g_loss: 6.842190742492676 - d_loss: 0.01163228414952755
[8835] Saved checkpoint: log/ckpts/ckpt-285
Epoch 285 completed.
[8840] g_loss: 7.600561141967773 - d_loss: 0.0023116362281143665
[8850] g_loss: 8.2756929397583 - d_loss: 0.014357846230268478
[8860] g_loss: 8.613699913024902 - d_loss: 0.002700233133509755
[8866] Saved checkpoint: log/ckpts/ckpt-286
Epoch 286 completed.
[8870] g_loss: 5.505533218383789 - d_loss: 0.02954746037721634
[8880] g_loss: 4.5686421394348145 - d_loss: 0.09842093288898468
[8890] g_loss: 9.45881462097168 - d_loss: 0.0012989076785743237
[8897] Saved checkpoint: log/ckpts/ckpt-287
Epoch 287 completed.
[8900] g_loss: 7.530735015869141 - d_loss: 0.02875472418963909
[8910] g_loss: 11.174363136291504 - d_loss: 0.03464796021580696
[8920] g_loss: 8.530645370483398 - d_loss: 0.002030392875894904
[8928] Saved checkpoint: log/ckpts/ckpt-288
Epoch 288 completed.
[8930] g_loss: 6.105060577392578 - d_los

## Towards Serving

The path to the production is straightforward. We only have to save the model parameters and export them as a graph.
Begin our generator a Keras model, saving it to the disk is as easy as calling the `save_weights` method in the Generator model.

In [9]:
G.save_weights('generator')

---
## Links

<a id="1">[1]</a>: Generative Adversarial Networks : https://arxiv.org/abs/1406.2661

<a id="2">[2]</a>: really-awesome-gan : https://github.com/nightrome/really-awesome-gan

<a id="3">[3]</a>: DCGAN : https://arxiv.org/abs/1511.06434

<a id="4">[4]</a>: Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift : https://arxiv.org/abs/1502.03167

<a id="5">[5]</a>: How Does Batch Normalization Help Optimization? : https://arxiv.org/abs/1805.11604